In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import sys
sys.path.append("./utils")

from generate_features import *
from get_targets import *
from dnn_utils import *

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt

Current device: cuda


In [2]:
DATA_PATH = "./data/prices/apple_1m_prices_train.csv"

In [3]:
data = pd.read_csv(DATA_PATH, index_col=0)
data

,v,vw,o,c,h,l,t,n
0,72572.0,67.6880,67.8325,67.6950,67.8325,67.5250,2020-03-09 08:00:00,89
1,23804.0,67.5453,67.9750,67.1250,67.9750,67.1250,2020-03-09 08:01:00,92
2,6024.0,67.2296,67.2500,67.1300,67.2500,67.1300,2020-03-09 08:02:00,27
3,2844.0,67.1575,67.1525,67.1550,67.1550,67.1525,2020-03-09 08:03:00,16
4,16712.0,67.1535,67.1575,67.1875,67.1875,67.0750,2020-03-09 08:04:00,34
...,...,...,...,...,...,...,...,...
25596,301724.0,141.6478,141.7000,141.7177,141.7200,141.5800,2021-09-30 16:16:00,2345
25597,170391.0,141.7013,141.7050,141.6550,141.7495,141.6300,2021-09-30 16:17:00,1273
25598,203458.0,141.6327,141.6500,141.6100,141.6900,141.5800,2021-09-30 16:18:00,1390
25599,230283.0,141.5619,141.6100,141.5599,141.6150,141.5000,2021-09-30 16:19:00,1603


In [4]:
indicators_data = lstm_indicators(data)
indicators_data.dropna(inplace=True)
indicators_data

,volume,open,close,high,low,volume_6_ema,volume_14_ema,volume_26_ema,volume_42_ema,volume_6_smma,...,boll,boll_ub,boll_lb,chop_14,chop_26,chop_42,mfi_6,mfi_14,mfi_26,mfi_42
1,23804.0,67.9750,67.1250,67.9750,67.1250,44124.000000,46446.285714,47250.153846,47607.428571,45971.272727,...,67.410000,68.216102,66.603898,11.700444,9.477357,8.261336,0.500000,0.500000,0.500000,0.500000
2,6024.0,67.2500,67.1300,67.2500,67.1300,26996.477064,31004.835314,32437.997043,33081.118458,30167.956044,...,67.316667,67.971978,66.661355,15.586256,12.624863,11.004992,0.500000,0.500000,0.500000,0.500000
3,2844.0,67.1525,67.1550,67.1550,67.1525,17667.310811,22389.641407,24164.808715,24973.528490,21372.166915,...,67.276250,67.835200,66.717300,16.317791,13.217406,11.521506,0.500000,0.500000,0.500000,0.500000
4,16712.0,67.1575,67.1875,67.1875,67.0750,17332.023388,20908.353451,22436.471634,23160.206556,20073.612556,...,67.258500,67.749030,66.767970,17.279556,13.996436,12.200580,0.500000,0.500000,0.500000,0.500000
5,4752.0,67.1875,67.1250,67.1875,67.1250,13187.264252,17170.061305,18894.413012,19715.628527,16234.198447,...,67.236250,67.688331,66.784169,18.911691,15.318465,13.352982,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322940,301724.0,141.7000,141.7177,141.7200,141.5800,278641.879596,246750.748582,228061.728778,216765.777072,255813.602469,...,141.798005,142.158626,141.437384,53.167959,50.711825,41.728999,0.472912,0.369512,0.383565,0.351824
322941,170391.0,141.7050,141.6550,141.7495,141.6300,247713.056854,236569.448771,223789.822942,214608.810696,241576.502057,...,141.785250,142.147220,141.423280,54.440544,50.700771,41.618304,0.458033,0.420070,0.424314,0.373698
322942,203458.0,141.6500,141.6100,141.6900,141.5800,235068.754896,232154.588935,222283.761984,214090.168338,235223.418381,...,141.760750,142.098543,141.422957,55.313477,50.143727,41.531162,0.504314,0.411365,0.387908,0.349270
322943,230283.0,141.6100,141.5599,141.6150,141.5000,233701.396354,231905.043743,222876.298133,214843.323299,234400.015318,...,141.733245,142.039403,141.427087,57.366395,50.022861,45.070049,0.509663,0.414892,0.395011,0.353011


In [5]:
indicators_data["target"] = get_direction_with_std_threshold_target(indicators_data, 30, 15, 3)
indicators_data.dropna(inplace=True)

In [6]:
X = indicators_data.drop(["target"], axis=1).to_numpy()
y = indicators_data["target"].to_numpy()

np.unique(y, return_counts=True)

(array([0., 1., 2.], dtype=float32),
 array([188973,  68678,  65262], dtype=int64))

In [11]:
TEST_RATIO = 0.2
X_lstm, X_catboost, y_lstm, y_catboost = train_test_split(X, y, test_size=0.5, shuffle=False)

X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_lstm, test_size=TEST_RATIO, shuffle=False)
X_lstm_train, X_lstm_val, y_lstm_train, y_lstm_val = train_test_split(X_lstm_train, y_lstm_train, test_size=TEST_RATIO, shuffle=False)

X_catboost_train, X_catboost_test, y_catboost_train, y_catboost_test = train_test_split(X_catboost, y_catboost, test_size=TEST_RATIO, shuffle=False)

In [12]:
train_dataset = StockSequenceDataset(torch.tensor(X_lstm_train), torch.tensor(y_lstm_train), 64)
val_dataset = StockSequenceDataset(torch.tensor(X_lstm_val), torch.tensor(y_lstm_val), 64)
test_dataset = StockSequenceDataset(torch.tensor(X_lstm_test), torch.tensor(y_lstm_test), 64)

train_loader = DataLoader(train_dataset, batch_size=50, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

In [13]:
class_weight = compute_class_weight(class_weight="balanced", classes=np.unique(y_lstm_train), y=y_lstm_train)

model = StockLSTMModel(input_size=X_lstm.shape[1], num_classes=3, sequence_length=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weight).float())

In [10]:
train_model(model, criterion, optimizer, train_loader, test_loader, 20)

////////////////////////////////////////
// Epoch: 1
// Train loss: 1.0874699208794572
// Validation loss: 1.0700967420231213
////////////////////////////////////////
////////////////////////////////////////
// Epoch: 2
// Train loss: 1.0722469324018897
// Validation loss: 1.060083811933344
////////////////////////////////////////
////////////////////////////////////////
// Epoch: 3
// Train loss: 1.0625523793988112
// Validation loss: 1.0630789019844749
////////////////////////////////////////
////////////////////////////////////////
// Epoch: 4
// Train loss: 1.066262835409583
// Validation loss: 1.085337151180614
////////////////////////////////////////
////////////////////////////////////////
// Epoch: 5
// Train loss: 1.0725335406094063
// Validation loss: 1.067427158355713
////////////////////////////////////////
////////////////////////////////////////
// Epoch: 6
// Train loss: 1.0631648011323882
// Validation loss: 1.0675778172232888
////////////////////////////////////////
//